In [126]:
from IPython.display import display, display_html, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
from threading import Thread
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Group Assignment
### Team Number: XX
### Team Member Names: Rehan, Rui, Anton
### Team Strategy Chosen: RISKY

*Delete before submission*

IMPORTANT SPECIFICATIONS DURING CODING:

- Stocks must be in US Market

- Stocks must have an average monthly volume of at least 200 000 shares between January 01, 2022 to October 31, 2022.

- Months must have at least 20 trading days

- Must have a minimum of 12 stocks and a maximum of 25 stocks

- If we choose n stocks for our portfolio, each stock must be minimum of (100/(2n))% of the portfolio when weighted by value (i.e., the overall value of the shares purchased in that particular stock) as of closing prices on November 25, 2022

- No individual stock may make up more than 25% of the portfolio when weighted by value (i.e., the overall value of the shares purchased in that particular stock) as of closing prices on November 26, 2022.

- Must spend all 500 000 USD on portfolio

- Teams will purchase their stocks at the closing prices on November 25, 2022

There are more specifications for how our code should be presented before submission, look into the assignment doc for info.

### Extracting tickers and adding them to a list

In [127]:
# Initializing a variable for our file name
file = 'Tickers_Example.csv'

# Initializing a start and end date for our portfolio
start_date = '2022-01-01'
end_date = '2022-11-01'

# Initializing a dataframe for 'raw' data extracted from the .csv file
tickers_raw = pd.read_csv(file, header=None)[0].tolist()

# Empty data structures to store ticker data in
tickers = []
tickers_hist = {}

# Function which consumes a ticker and determines the validation based on prerequisites
def validate_ticker(ticker):

    # Extracting ticker info from yFinance
    ticker_info = yf.Ticker(ticker).info

    # Trying every stock and excepting those that throw an error
    try:
        # If the stock is valid, we check for each prerequisite:
        # Checking for USD currency and ensuring it's on the US market
        if ticker_info['currency'] == 'USD' and ticker_info['market'] == 'us_market':
            ticker_hist = yf.Ticker(ticker).history(start=start_date, end=end_date, interval='1d').dropna()

            # Checking monthly volume
            ticker_monthly_trading_days = ticker_hist['Volume'].groupby(pd.Grouper(freq='MS')).count()
            ticker_monthly_volume = ticker_hist['Volume'].groupby(pd.Grouper(freq='MS')).sum()

            # Checking if the month has at least 20 trading days
            for month in ticker_monthly_trading_days.index:
                if ticker_monthly_trading_days.loc[month] < 20:
                    ticker_monthly_volume.drop(month, inplace=True)

            # Checking if the average monthly volume is greater than or equal to 200,000 USD
            if ticker_monthly_volume.mean() >= 200000:
                tickers.append(ticker)
                tickers_hist[ticker] = ticker_hist
            else:
                print(f'{ticker} Ticker does not meet average monthly volume requirements')
        else:
            print(f'{ticker} Ticker does not reference stock denominated in USD')
    except:
        print(f'Error: {ticker} Ticker does not reference a valid stock')

# Empty data structure for threading
threads = []

# Checking validity of each ticker in list of tickers given from threading
for ticker in tickers_raw:
    thread = Thread(target=validate_ticker, args=[ticker])
    thread.start()
    threads.append(thread)

# Using threading
for thread in threads:
    thread.join()

Error: PCLN Ticker does not reference a valid stock
Error: AGN Ticker does not reference a valid stock
Error: TWX Ticker does not reference a valid stock
Error: RTN Ticker does not reference a valid stock
Error: CELG Ticker does not reference a valid stock
TD.TO Ticker does not reference stock denominated in USD
RY.TO Ticker does not reference stock denominated in USD


### List of Valid Tickers

In [128]:
print(tickers)

['BIIB', 'BMY', 'GOOG', 'BK', 'MRK', 'C', 'CAT', 'BLK', 'NKE', 'NEE', 'UPS', 'MO', 'CVS', 'AIG', 'CSCO', 'COP', 'GM', 'CL', 'ABT', 'PFE', 'MSFT', 'AAPL', 'T', 'OXY', 'UNH', 'COF', 'TXN', 'PYPL', 'LLY', 'MON', 'SBUX', 'SO', 'KMI', 'CMCSA', 'USB', 'QCOM', 'JPM', 'PM', 'BA', 'COST', 'BAC', 'AXP', 'ORCL', 'ACN', 'PG', 'TGT', 'LMT', 'ABBV', 'KO', 'UNP', 'AMZN', 'MS', 'PEP', 'SLB', 'SPG']


---

## Strategy and Data Analytics

The strategy our team has chosen is to go risky, meaning we must optimize a portfolio so that it is driven away as much as possible, from the initial starting value of $500,000. Since we are pursuing a risky strategy, there are a few factors to an optimal portfolio.

Since we want the most risk and most reward, we want to minimize any diversification involved. Essentially, out of the minimum and maximum requirements for stock diversification, we want to pick the least amount of stocks, which is a minimum of 12 and keep diversification at an all time low. Furthermore, our stocks can net heavy volume, however, volume is useless if the market direction for these assets is all over the place. Therefore, we need all of our stocks to move in the same direction. 

Overall, we will move forward with the minimum amount of stocks involved, 12, and keep track of various factors like risk to reward ratio, positive risk, negative risk, betas and options markets.

### Measuring Beta

Beta calculates the volatility of a stock given the covariance of the stock relative to the broader stock market and the variance of the stock. We can use this to our advantage and find the higher volatility stocks so that we can create a riskier portfolio. The formula for Beta is as follows:

$$
\beta_i = \frac {\mathrm{Cov} (r_i,r_m)}{\mathrm{Var} (r_m)}
$$


Where $ \beta_i $ is the market beta of a stock, $ {\mathrm{Cov} (r_i,r_m)} $ is the covariance between the stock and the market index (in our case, the S&P 500), $ {\mathrm{Var} (r_m)} $ is the variance of the market index. Since a volatile stock can net us a gain or a loss, we want the highest magnitude. Thus, we will calculate both an upper bound beta and a lower bound beta and yield the highest magnitude out of the two as a contender for our final portfolio

In [129]:
# We will be using S&P 500 as our measure of how the overall stock market is performing
sp_index = yf.Ticker("^GSPC") # ticker for S&P 500

# Extracting close prices of the S&P 500, percentage returns and its corresponding variance
sp_hist = sp_index.history(start=start_date, end=end_date).filter(like="Close")
sp_hist["Returns"] = sp_hist['Close'].pct_change()
sp_var = sp_hist["Returns"].var()

In [130]:
frame = {}
beta_values = []
min_stocks = 12
max_stocks = 25

# calculate_betas consumes a list of tickers then creates a dataframe of the tickers and their respective betas
def calculate_betas(tickers):
    for i in range(len(tickers)):
        # Getting the ticker and calculating its returns
        ticker_hist = tickers_hist[tickers[i]]
        ticker_close = ticker_hist["Close"].pct_change()

        # Initializing temporary dataframe with 2 columns containing the ticker's returns and S&P 500's returns
        frame = {tickers[i]: ticker_close,
                 "S&P500": sp_hist['Returns']}
        temp_dataframe = pd.DataFrame(frame)

        # Calculating Beta and appending onto a list of betas
        beta = temp_dataframe.cov() / sp_var
        beta_values.append(beta.iat[0,1])

    return beta_values

# Storing the tickers with their respective betas in a dataframe 
beta_frame = {"Ticker": tickers,
              "Beta": calculate_betas(tickers)}
beta_dataframe = pd.DataFrame(beta_frame)

# Sorting the betas in both ascending and descending order
beta_ascending = beta_dataframe.sort_values("Beta", ascending = True)
beta_descending = beta_dataframe.sort_values("Beta", ascending = False)

---

### The stocks with the lowest betas:

In [131]:
beta_ascending.iloc[:min_stocks]

,Ticker,Beta
29,MON,0.010761
46,LMT,0.309023
4,MRK,0.313415
1,BMY,0.319430
47,ABBV,0.338508
11,MO,0.359057
17,CL,0.377967
37,PM,0.407042
31,SO,0.414797
44,PG,0.466520


### The stocks with the highest betas:

In [132]:
beta_descending.iloc[:min_stocks]

,Ticker,Beta
27,PYPL,1.651318
50,AMZN,1.604552
35,QCOM,1.454487
16,GM,1.393548
2,GOOG,1.310727
8,NKE,1.300717
38,BA,1.298421
7,BLK,1.269739
21,AAPL,1.265965
20,MSFT,1.247844


---

### Optimal Weighting for Stocks

Since we want the highest volatility possible, we will invest the majority of our funds into the most volatile stock(s) before investing into the others. Given the requirements of a stock only being a maximum of 25% of the entire portfolio, we have calculated the optimal weighting below to create the most volatility.

In [133]:
min_weighting = 100 / (min_stocks * 2)
max_weighting = 25
list_of_weighting = []
total_weighting = 0 #total_weighting will need to add up to 100 in the end
num_stocks = min_stocks


# We want to optimize the weighting of each stock such that the more volatile stocks take up the majority of the portfolio
while (num_stocks != 0):
    temp_weighting = min_weighting * (num_stocks - 1)
    remaining_weight = 100 - (temp_weighting + total_weighting)

    if (remaining_weight > max_weighting):
        list_of_weighting.append(max_weighting)
        total_weighting += max_weighting
    elif (remaining_weight > min_weighting):
        list_of_weighting.append(remaining_weight)
        total_weighting += remaining_weight
    else:
        list_of_weighting.append(min_weighting)
        total_weighting += min_weighting
    num_stocks -= 1

---

## Sharpe Ratio vs Sortino Ratio

Let's consider the Sortino Ratio. One might wonder what the difference is between both ratios.

The Sharpe Ratio measures risk-adjusted return by comparing the return of an investment with its risk. The formula for the Sharpe Ratio is: <br>

$\begin{align}\large S_a = \frac{ R_p - R_f }{\sigma_o} \end{align}$

where $\normalsize R_p$ is expected return, $\normalsize R_f$ is the risk-free return rate and $\normalsize \sigma_o$ is the standard deviation of the excess returns.

On the other hand, the Sortino Ratio also measures risk-adjusted return. However, the formula for the Sortino Ratio is: <br>

$\begin{align}\large S_a = \frac{ R_p - R_f }{\sigma_d} \end{align}$

where $\normalsize R_p$ is expected return, $\normalsize R_f$ is the risk-free return rate and $\normalsize \sigma_d$ is the standard deviation of the negative or downside returns.

Although both equations are **almost** identical, the difference is that the Sharpe Ratio dislikes volatility as it accounts for any excess returns, whereas the Sortino Ratio accounts for any negative or downside returns. Consequently, graphed sharpe ratios would produce a graph that would more likely be linear, contrary to a more exponential graph of Sortino Ratios.

The Sharpe ratio is more useful when evaluating low-volatility investment portfolios, whereas the Sortino ratio is more useful when evaluating high-volatility investment portfolios. As we are looking to maximize our risk on our portfolio, it would be a good idea to assess both the Sortino ratio and the Sharpe ratio.

### Sharpe Ratio

In [134]:
# Creating empty dataframes to store data in
sharpe = pd.DataFrame()
data = pd.DataFrame()
sharpe_ratios = []

# sharpe['Total Value'] = data['Total Value']
# sharpe['Total Value Percentage Change'] = data['Total Value'].pct_change()

sharpe_ratio = data.pct_change().mean() / data.pct_change().std()
print(sharpe_ratio)







Series([], dtype: float64)


## Risk Coefficient

To further maximize risk from our selection of tickers, we can rearrange our ratios to solve for risk coefficient.


## Options Market

Another factor to look into is the options market. Although the options market doesn't directly tell us how a stock is performing, the option traders may give us a hint as to where the stock is heading. If the ratio of puts to calls is high, the stock price will most likely drop. If the ratio of calls to puts is high, the stock price will most likely jump.

### Put to Call Ratio ###
- Divide sum of puts by sum of calls
- More calls than puts means the asset price will increase (as investors want to buy the asset at a predetermined price) --> they are expecting the price to increase
- More puts than calls means the asset price will decrease (as investors want to sell the asset at a predetermined price) --> they are expecting the price to fall

In [143]:
options_evaluation_raw = {}
options_evaluation_raw['puts_sum'] = {}
options_evaluation_raw['calls_sum'] = {}
options_evaluation_raw['pcr'] = {}
options_evaluation_raw['cpr'] = {}

def extract_options_market(ticker):

    try:
        asset = yf.Ticker(ticker)
        options = asset.option_chain(asset.options[0])

        puts = pd.DataFrame().append(options.puts)
        puts_sum = puts['volume'].sum()
        options_evaluation_raw["puts_sum"][ticker] = puts_sum

        calls = pd.DataFrame().append(options.calls)
        calls_sum = calls['volume'].sum()
        options_evaluation_raw["calls_sum"][ticker] = calls_sum

        puts_to_calls =  puts_sum / calls_sum
        options_evaluation_raw["pcr"][ticker] = puts_to_calls

        calls_to_puts =  calls_sum / puts_sum
        options_evaluation_raw["cpr"][ticker] = calls_to_puts
    
    except IndexError as error:
        print(f'Dropped {ticker}, no options data available')
    
# Checking options market of each ticker in list of tickers given from threading
for ticker in tickers:
    thread = Thread(target=extract_options_market, args=[ticker])
    thread.start()
    threads.append(thread)

# Using threading
for thread in threads:
    thread.join()        


Dropped MON, no options data available


In [165]:
if len(options_evaluation_raw.keys()) <= 12:
    options_evaluation = pd.DataFrame(options_evaluation_raw)
    print("The options market was large enough for at least 12 stocks. Here is the summary:")
    display(options_evaluation.head())
else:
    print("There was insufficient options data for the tickers provided.")

The options market was large enough for at least 12 stocks. Here is the summary:


,puts_sum,calls_sum,pcr,cpr
BK,12.0,21.0,0.571429,1.750000
MO,567.0,228.0,2.486842,0.402116
PM,60.0,275.0,0.218182,4.583333
USB,257.0,64.0,4.015625,0.249027
BMY,297.0,190.0,1.563158,0.639731


Let's filter for any stocks with a put-to-call ratio which is below one, and filter for any stocks with a call-to-put ratio which is below one separately. We want our portfolio to move large in one direction, so we must account for stocks which may offset any gains and losses from other stocks. Thus, we will create a portfolio strictly consisting of heavy put-to-call ratio stocks and another portfolio strictly consisting of heavy call-to-put ratio stocks for further comparison. If only one portfolio can be formed, then the comparison step will be skipped.

In [167]:
# Filtering for stocks with a put-to-call ratio greater than 1 and sorting
pcr_filter = pd.DataFrame()
pcr_filter = options_evaluation[options_evaluation["pcr"] > 1].dropna()
pcr_filter = pcr_filter.sort_values(by=['pcr'], ascending=False)
pcr_filter.head()

,puts_sum,calls_sum,pcr,cpr
BLK,454.0,408.0,1.112745,0.898678
KO,440.0,384.0,1.145833,0.872727
UNH,190.0,158.0,1.202532,0.831579
ABT,2370.0,1877.0,1.262653,0.791983
JPM,646.0,467.0,1.383298,0.722910


Forming the portfolio for the heavy put-to-call ratio stocks,

In [182]:
# Checking if there are enough heavy put-to-call ratio stocks
if len(pcr_filter) >= 12:
    print("There are at least 12 stocks with a heavy put-to-call ratio.")
    pcr_heavy_tickers = []
    pcr_heavy_tickers = list(pcr_filter.index.values)
    # Removing everything after the first 12 tickers
    pcr_heavy_tickers = pcr_heavy_tickers[0:12]
    print(pcr_heavy_tickers)
else: 
    print("There are not enough stocks with a put-to-call ratio greater than 1.")


There are at least 12 stocks with a heavy put-to-call ratio.
['BLK', 'KO', 'UNH', 'ABT', 'JPM', 'COST', 'MRK', 'BMY', 'SPG', 'MO', 'CAT', 'NKE']


In [168]:
# Filtering for stocks with a call-to-put ratio greater than 1
cpr_filter = pd.DataFrame()
cpr_filter = options_evaluation[options_evaluation["cpr"] > 1].dropna()
cpr_filter = cpr_filter.sort_values(by=['cpr'], ascending=False)
cpr_filter.head()

,puts_sum,calls_sum,pcr,cpr
AIG,193.0,4247.0,0.045444,22.005181
NEE,443.0,6556.0,0.067572,14.799097
MS,3945.0,24982.0,0.157914,6.332573
PM,60.0,275.0,0.218182,4.583333
SBUX,182.0,778.0,0.233933,4.274725


In [183]:
# Checking if there are enough heavy put-to-call ratio stocks
if len(cpr_filter) >= 12:
    print("There are at least 12 stocks with a heavy put to call ratio.")
    cpr_heavy_tickers = []
    cpr_heavy_tickers = list(cpr_filter.index.values)
    # Removing everything after the first 12 tickers
    cpr_heavy_tickers = cpr_heavy_tickers[0:12]
    print(cpr_heavy_tickers)
else: 
    print("There are not enough stocks with a call-to-put ratio greater than 1.")


There are at least 12 stocks with a heavy put to call ratio.
['AIG', 'NEE', 'MS', 'PM', 'SBUX', 'PFE', 'SO', 'COF', 'OXY', 'C', 'BAC', 'UPS']


#### Below is the function that calculates the number of shares we will invest in each stock given a list of tickers ####
So far, we have obtained the tickers from the methods we used above and gotten a couple of contestents for our final portfolio. Now, we will be throwing them into the function below to yield the amount of stocks we will invest into each ticker for each portfolio.

In [141]:
# CHANGE HERE -- put investment at the top so the user can change it as desired
investment = 500000

# This function calculates the number of stocks to invest in given a list of 12 stocks
def num_stocks (tickers_list):

    # Calculating the $ amount to invest into each stock 
    investment_list = []
    for i in range(len(list_of_weighting)):
        investment_list.append((list_of_weighting[i] * investment))

    # Getting the number of stocks invested in each ticker
    stocks_in_each = []
    for i in range(len(tickers_list)):
        stocks_in_each.append (tickers_hist.get(tickers_list[i])['Close'][0])

    frame = {"Ticker": tickers_list,
             "Stocks": stocks_in_each,
             "Investment": investment_list}

    stocks_dataframe = pd.DataFrame(frame)

    return (stocks_dataframe)

### Checking if the options market is large enough for the chosen stocks:

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.